#### Load Libraries and Demo Codes

In [ ]:
import julia; julia.install(quiet=True)
from julia import Main

import numpy as np
import sympy as sp

import plotly.graph_objects as go
from scipy.spatial.transform import Rotation as R

import holoviews as hv
from holoviews import opts
import panel as pn
import param

pn.extension('katex', 'plotly')
hv.extension('bokeh')

%load_ext julia.magic

In [ ]:
%%julia
using Pkg; Pkg.activate("../GenLinAlgProblems")
using GenLinAlgProblems, LinearAlgebra, Printf, Latexify, LaTeXStrings, Random, SymPy

In [ ]:
#%output backend="bokeh"
class VectorAngleViewer(pn.viewable.Viewer):
    #angle_times_10 = param.Number(default=450, bounds=(0, 1800), label="Angle x10 (degrees)")
    angle          = param.Number(default=45, bounds=(0, 180), label="Angle (degrees)")

    def __init__(self, **params):
        super().__init__(**params)
        self.plot_pane = pn.pane.HoloViews(height=400, width=400)
        self.text_pane = pn.pane.LaTeX(width=300, renderer='katex')
        self._update_plot()
        #self.param.watch(self._update_plot, ['angle_times_10'])
        self.param.watch(self._update_plot, ['angle'])

    def _update_plot(self, *_):
        #angle_rad = np.radians(self.angle_times_10*0.1)
        angle_rad = np.radians(self.angle)
        u = np.array([1.0, 0.0])
        v = np.array([np.cos(angle_rad), np.sin(angle_rad)])
        dot_product = np.dot(u, v)

        segments = hv.Segments([
            (0, 0, u[0], u[1]),
            (0, 0, v[0], v[1])
        ]).opts(color='blue', line_width=3)

        arc_radius = 0.5
        angle_range = np.linspace(0, angle_rad, 100)
        arc_x = arc_radius * np.cos(angle_range)
        arc_y = arc_radius * np.sin(angle_range)
        arc = hv.Curve((arc_x, arc_y)).opts(color='darkred', line_width=2)

        #annotation = hv.Text(0.5, 1.1, f"{0.1*self.angle_times_10:.1f}°").opts(color='black', text_align='center', text_baseline='bottom')
        annotation = hv.Text(0.5, 1.1, f"{self.angle:.1f}°").opts(color='black', text_align='center', text_baseline='bottom')

        line = hv.HLine(0).opts(color='#989F7A', line_width=1.5)

        plot = (line * segments * arc * annotation).opts(
            width=400, height=400, xlim=(-1.5, 1.5), ylim=(0, 1.5),
            aspect='equal', title="Angle Between Two Vectors"
        )

        #latex = self._latex_text(v[0], v[1], dot_product, 0.1*self.angle_times_10)
        latex = self._latex_text(v[0], v[1], dot_product, self.angle)
        self.plot_pane.object = plot
        self.text_pane.object = latex

    def _latex_text(self, vx, vy, dot, angle):
        return (
            r"$\Large \begin{aligned}"
            r"u &= \begin{bmatrix} 1.00 \\ 0.00 \end{bmatrix} \\ "
            rf"v &= \begin{{bmatrix}} {vx:.2f} \\ {vy:.2f} \end{{bmatrix}} \\ "
            rf"u \cdot v &= {dot:.2f} \\ \\ \ "
            rf"\theta &= {angle:.2f}^\circ"
            r"\end{aligned}$"
        )

    def __panel__(self):
        return pn.Row(
            self.plot_pane,
            pn.Spacer(width=30),
            pn.Column(
                pn.Spacer(height=20),
                self.text_pane,
                pn.Spacer(height=20),
                pn.widgets.FloatSlider.from_param(self.param.angle, width=300),
                #pn.widgets.Player.from_param(self.param.angle_times_10, width=300, interval=120, step=2, start=0, end=1800, loop_policy='reflect'),
            )
        )

In [ ]:
#%output backend="plotly"
def spherical_to_cartesian(r, theta_deg, phi_deg):
    theta = np.radians(theta_deg)
    phi = np.radians(phi_deg)
    return r * np.array([
        np.sin(theta) * np.cos(phi),
        np.sin(theta) * np.sin(phi),
        np.cos(theta)
    ])
def project_to_plane(v, normal):
    n = normal / np.linalg.norm(normal)
    return v - np.dot(v, n) * n
class VectorProjectionDemo(pn.viewable.Viewer):
    theta = param.Integer(default=60, bounds=(-90, 90), label="Theta (from z)")
    phi   = param.Integer(default=45, bounds=(0, 360), label="Phi (around z)")

    def __init__(self, **params):
        super().__init__(**params)
        self.plot_pane = pn.pane.Plotly(height=600, config={'responsive': True})
        self._update_plot()
        self.param.watch(self._update_plot, ['theta', 'phi'])
    def build_traces(self, r=4):
        theta_deg = self.theta
        phi_deg = self.phi
        v = spherical_to_cartesian(r, theta_deg, phi_deg)
        normal = np.array([0, 0, 1])
        proj = project_to_plane(v, normal)

        angle_rad = np.arccos(np.clip(np.linalg.norm(proj) / np.linalg.norm(v), -1, 1))
        angle_deg = np.round(np.degrees(angle_rad), 2)

        vector = go.Scatter3d(
            x=[0, v[0]], y=[0, v[1]], z=[0, v[2]],
            mode='lines+markers',
            line=dict(color='darkred', width=6),
            marker=dict(size=5, color='darkred'),
            name='Vector v'
        )

        leg1 = go.Scatter3d(
            x=[0, proj[0]], y=[0, proj[1]], z=[0, proj[2]],
            mode='lines',
            line=dict(color='black', width=4),
            name='Origin → proj(v)'
        )
        leg2 = go.Scatter3d(
            x=[proj[0], v[0]], y=[proj[1], v[1]], z=[proj[2], v[2]],
            mode='lines',
            line=dict(color='black', width=4),
            name='proj(v) → v'
        )

        xx, yy = np.meshgrid(np.linspace(-4, 4, 2), np.linspace(-4, 4, 2))
        zz = np.zeros_like(xx)
        plane = go.Surface(
            x=xx.tolist(), y=yy.tolist(), z=zz.tolist(),
            opacity=0.85,
            showscale=False,
            colorscale=[[0, '#989F7A'], [1, '#989F7A']],
            name='Plane'
        )
        return [plane, vector, leg1, leg2], angle_deg
    def _update_plot(self, *_):
        try:
            current_camera = self.plot_pane.object['layout']['scene']['camera']
        except Exception:
            current_camera = dict(eye=dict(x=2.0, y=0.65, z=0.30))

        traces, angle_deg = self.build_traces()
        fig = go.Figure(data=traces)
        fig.update_layout(
            scene=dict(
                xaxis=dict(range=[-5, 5]),
                yaxis=dict(range=[-5, 5]),
                zaxis=dict(range=[-1, 5]),
                aspectmode='cube',
                camera=current_camera
            ),
            margin=dict(l=0, r=0, t=30, b=50),
            title=f"Vector Projection — Angle: {angle_deg:.2f}°",
            legend=dict(orientation="h", yanchor="top", y=-0.2, xanchor="center", x=0.5, font=dict(size=12))
        )
        self.plot_pane.object = fig
    def __panel__(self):
        return pn.Row(
            self.plot_pane,
            pn.Column(
                "## Vector–subspace Angle: Shortest Angle to the Plane.",
                pn.Spacer(height=100),
                pn.widgets.Player.from_param(self.param.theta, width=250, interval=30, step=2, loop_policy='reflect'),
                pn.Spacer(height=10),
                pn.widgets.Player.from_param(self.param.phi, width=250, interval=30, step=3, loop_policy='reflect'),
            )
        )

In [ ]:
#%output backend="plotly"
def create_plane(origin, v1, v2, size=2.0):
    v1 = v1 / np.linalg.norm(v1)
    v2 = v2 / np.linalg.norm(v2)
    points = [origin + a * v1 + b * v2 for a in [-size, size] for b in [-size, size]]
    x = np.array([p[0] for p in points]).reshape(2, 2)
    y = np.array([p[1] for p in points]).reshape(2, 2)
    z = np.array([p[2] for p in points]).reshape(2, 2)
    return x, y, z
def plot_vector_pair(u_vec, v_vec, scale=1.6, u_color='darkred', v_color='blue', name_prefix='principal'):
    return [
        go.Scatter3d(x=[0, u_vec[0]*scale], y=[0, u_vec[1]*scale], z=[0, u_vec[2]*scale],
                     mode='lines', line=dict(color=u_color, width=6), name=f'{name_prefix}_u'),
        go.Scatter3d(x=[0, v_vec[0]*scale], y=[0, v_vec[1]*scale], z=[0, v_vec[2]*scale],
                     mode='lines', line=dict(color=v_color, width=6), name=f'{name_prefix}_v')
    ]
def angle_arc(a, b, origin=np.zeros(3), steps=30, scale=1.6, color='green', name='Angle Arc'):
    a = a / np.linalg.norm(a)
    b = b / np.linalg.norm(b)
    axis = np.cross(a, b)
    if np.linalg.norm(axis) < 1e-6:
        return None
    axis = axis / np.linalg.norm(axis)
    angle = np.arccos(np.clip(np.dot(a, b), -1.0, 1.0))
    pts = [R.from_rotvec(axis * t).apply(a) * scale for t in np.linspace(0, angle, steps)]
    pts = np.array(pts) + origin
    return go.Scatter3d(
        x=pts[:, 0], y=pts[:, 1], z=pts[:, 2],
        mode='lines', line=dict(color=color, width=4, dash='dot'),
        name=name
    )
def plot_intersection_line(u1, u2, v1, v2, scale=2.5):
    nU = np.cross(u1, u2)
    nV = np.cross(v1, v2)
    line_dir = np.cross(nU, nV)
    if np.linalg.norm(line_dir) < 1e-6:
        return None
    line_dir /= np.linalg.norm(line_dir)
    return go.Scatter3d(
        x=[-scale*line_dir[0], scale*line_dir[0]],
        y=[-scale*line_dir[1], scale*line_dir[1]],
        z=[-scale*line_dir[2], scale*line_dir[2]],
        mode='lines', line=dict(color='black', width=3, dash='dash'),
        name='Intersection Line'
    )

class PrincipalAnglePlayer(pn.viewable.Viewer):
    offset = param.Number(default=0.0,  bounds=(-5.0, 5.0), label="Z Offset for Plane V")
    theta_1 = param.Integer(default=0,  bounds=(-1800, 1800), label="theta_1")  # 10*degrees
    theta_2 = param.Integer(default=90, bounds=(-1800, 1800), label="theta_2")  # 10*degrees

    r     = param.Number(default=4.5, bounds=(1.0, 10.0), label="Camera Distance")
    theta = param.Number(default=45,  bounds=(0, 360),    label="Azimuth (θ°)")
    phi   = param.Number(default=30,  bounds=(-90, 90),   label="Elevation (φ°)")
    use_svd = param.Boolean(default=False, doc="Use SVD to compute principal vectors")

    def __init__(self, **params):
        super().__init__(**params)
        self.fig_pane = pn.pane.Plotly(height=650, config={'responsive': True})
        self.param.watch(self._update_data_only, ['theta_1', 'theta_2', 'offset', 'use_svd'])
        self.param.watch(self._update_camera_only, ['r', 'theta', 'phi'])
        self._current_camera = self._camera_eye()
        self._update_data_only()
    def _svd_from_planes(self, u1, u2, v1, v2):
        U, _ = np.linalg.qr(np.stack([u1, u2], axis=1))
        V, _ = np.linalg.qr(np.stack([v1, v2], axis=1))
        return np.linalg.svd(U.T @ V), U, V
    def compute_principal_vectors_and_angles(self, u1, u2, v1, v2):
        (W1, s, W2T), U, V = self._svd_from_planes(u1, u2, v1, v2)
        principal_U = U @ W1
        principal_V = V @ W2T.T
        angles = np.arccos(np.clip(s, -1, 1))
        return principal_U, principal_V, np.degrees(angles)
    def compute_geometric_principal_vectors(self, theta1_rad, theta2_rad):
        p1 = np.array([np.cos(theta1_rad), np.sin(theta1_rad), 0])
        perp = np.array([-np.sin(theta1_rad), np.cos(theta1_rad), 0])
        p2_v = np.cos(theta2_rad) * perp + np.sin(theta2_rad) * np.array([0, 0, 1])
        p2_v = p2_v / np.linalg.norm(p2_v)
        p2_u = perp / np.linalg.norm(perp)
        principal_U = np.stack([p1, p2_u], axis=1)
        principal_V = np.stack([p1, p2_v], axis=1)
        angles = np.arccos([np.clip(np.dot(p1, p1), -1, 1), np.clip(np.dot(p2_u, p2_v), -1, 1)])
        return principal_U, principal_V, np.degrees(angles)
    def _camera_eye(self):
        θ, φ = np.radians(self.theta), np.radians(self.phi)
        x = self.r * np.cos(θ) * np.cos(φ)
        y = self.r * np.sin(θ) * np.cos(φ)
        z = self.r * np.sin(φ)
        return dict(x=x, y=y, z=z)
    def _update_camera_only(self, *_):
        self._current_camera = self._camera_eye()
        if self.fig_pane.object:
            self.fig_pane.object.update_layout(scene_camera=dict(eye=self._current_camera))
    def _update_data_only(self, *_):
        θ1 = np.radians(self.theta_1 * 0.1)
        θ2 = np.radians(self.theta_2 * 0.1)

        u1 = np.array([1, 0, 0])
        u2 = np.array([0, 1, 0])
        origin = np.zeros(3)

        v1 = np.array([np.cos(θ1), np.sin(θ1), 0])
        v1_perp = np.array([-np.sin(θ1), np.cos(θ1), 0])
        v2 = np.cos(θ2) * v1_perp + np.sin(θ2) * np.array([0, 0, 1])

        xU, yU, zU = create_plane(origin, u1, u2)
        xV, yV, zV = create_plane(origin + np.array([0, 0, self.offset]), v1, v2)

        if self.use_svd:
            principal_U, principal_V, angles = self.compute_principal_vectors_and_angles(u1, u2, v1, v2)
        else:
            principal_U, principal_V, angles = self.compute_geometric_principal_vectors(θ1, θ2)

        traces = [
            go.Surface(x=xU, y=yU, z=zU, opacity=0.85, showscale=False,
                       colorscale=[[0, '#989F7A'], [1, '#989F7A']], name="Plane U"),
            go.Surface(x=xV, y=yV, z=zV, opacity=0.65, showscale=False,
                       colorscale=[[0, '#bbbbbb'], [1, '#bbbbbb']], name="Plane V")
        ]

        traces += plot_vector_pair(principal_U[:, 0], principal_V[:, 0], name_prefix='p1')
        traces += plot_vector_pair(principal_U[:, 1], principal_V[:, 1], name_prefix='p2')

        arc1 = angle_arc(principal_U[:,0], principal_V[:,0], origin=origin, color='black', name='θ₁ arc')
        arc2 = angle_arc(principal_U[:,1], principal_V[:,1], origin=origin, color='black', name='θ₂ arc')
        if arc1:
            traces.append(arc1)
        if arc2:
            traces.append(arc2)

        inter_line = plot_intersection_line(u1, u2, v1, v2)
        if inter_line:
            traces.append(inter_line)

        title_text = f"Principal Angles: θ₁ = {angles[0]:.2f}°, θ₂ = {angles[1]:.2f}°"

        fig = go.Figure(data=traces)
        fig.update_layout(
            title=title_text,
            scene=dict(
                aspectmode='cube',
                xaxis=dict(tickmode='array', tickvals=[-2, -1, 0, 1, 2], range=[-2.5, 2.5], title='X'),
                yaxis=dict(tickmode='array', tickvals=[-2, -1, 0, 1, 2], range=[-2.5, 2.5], title='Y'),
                zaxis=dict(tickmode='array', tickvals=[-1, 0, 1, 2, 3], range=[-1.5, 3.0], title='Z'),
                camera=dict(eye=self._current_camera, projection=dict(type="orthographic"))
            ),
            legend=dict(orientation="h", yanchor="top", y=-0.2, xanchor="center", x=0.5, font=dict(size=12)),
            margin=dict(l=0, r=0, t=50, b=0),
            showlegend=True
        )
        self.fig_pane.object = fig
    def __panel__(self):
        return pn.Row(
            pn.Column(
                pn.pane.Markdown("## Principal Angle Visualizer (Intersection Line + Arcs + Projections)"),
                self.fig_pane,
            ),
            pn.Column(
                r"## Principal Angles: θ₁ (Intersection), θ₂ (Inclination).",
                pn.Spacer(height=40),
                "theta_1", pn.widgets.Player.from_param(self.param.theta_1, width=250, interval=10, step=5, start=-1800, end=1800, loop_policy='reflect'),
                "theta_2", pn.widgets.Player.from_param(self.param.theta_2, width=250, interval=10, step=5, start=-1800, end=1800, loop_policy='reflect'),
                pn.Spacer(height=30),
                pn.widgets.Checkbox.from_param(self.param.use_svd, name="Use SVD"),
            ),
        )

<div style="float:center;width:100%;text-align:center;">
<strong style="height:100px;color:darkred;font-size:40px;">Hyperplanes and Principal Angles</strong>
</div>

This notebook is the first in a series exploring principal angles between subspaces.<br>
$\qquad$ **Principal angles** offer a fundamental framework for comparing the **orientation and distance between subspaces,**<br>
$\qquad$ with applications spanning signal processing, dimensionality reduction, data clustering, and numerical linear algebra.

**Roadmap:**<br>
* review orthogonal projections
* define principal angles between subspaces
* explore basic properties through simple examples.

# 1. Foundations: Orthogonal Projections in Practice

Since subspace angles are based on projections, we first quickly review properties of orthogonal projections.<br>
Links to notebooks discussing orthogonality and orthogonal projections
* [**20_LengthOrthogonality.ipynb**](20_LengthOrthogonality.ipynb)
* [**Orthogonality.ipynb**](Orthogonality.ipynb)
* [**Orthogonal_Decomposition_Example.ipynb**](Orthogonal_Decomposition_Example.ipynb)
* [**NormalEquation_3_ways.ipynb**](NormalEquation_3_ways.ipynb)

____
To define the angles between two subspaces, we will make use of **orthogonal projections**.<br>
We begin by restating some basic facts.

**Reminder:**
* **Projection Matrices** $P$ are defined by the property $P^2 = P$.
* **Orthogonal Projection Matrices** further satisfy $P^T = P$.
* Given an **orthonormal basis** $B = \{\ q_1, q_2, \dots q_k\ \}$ for some subspace $S = \text{span}\ B,$<br>
an orthogonal projection matrix onto this subspace is given by $P = Q Q^T$, where $Q = \begin{pmatrix} q_1 & q_2 & \dots q_k \end{pmatrix}$

In [ ]:
%%julia
Random.seed!(1212)
Q = Q_matrix(3,general=true)[:,1:2]
display(py_show("Example: Consider the plane defined by the basis vectors ", L"\;\;q_1 =", Q[:,1], L",\;\;q_2 =", Q[:, 2]))
P = Q*Q'
py_show( L"\therefore \;\; Q = ", Q, L"\qquad P = Q Q^T =", P )

* The **eigenvalues** of a projection matrix $P$ are 0 and 1
* The **trace** of a projection matrix is thus the dimension of the subspace it projects onto

In [ ]:
%%julia
py_show("The trace of P is the dimension of the projection subspace: tr(P) = ", tr(P))

____

#### Computation

Given a matrix $A$, we can compute the orthogonal projection matrix $P$ onto the column space $\mathscr{C}(A)$ in a number of ways. Let $\tilde{A}$ be a submatrix of $A$ obtained by removing free variable columns if any. Then

<div style="float:left;margin-left:1cm;">

| Decomposition method of $A\qquad$ | Orthogonal projection onto $\mathscr{C}(A)\quad$ | Common use case |
| ---------| ----------- | :----------- |
| normal equation solution | $P = \tilde{A} \left( \tilde{A}^T \tilde{A} \right)^{-1} \tilde{A}^T$. | Efficient if $A$ is small or sparse |
| $Q R$ decomposition | $P = Q Q^T$. | Numerically stabel |
| reduced SVD $A = U_r \Sigma_r V_r^T$ | $P = U_r U_r^T$. | Explicitely highlights rank and subspace structure |
</div>

In [ ]:
%%julia
Random.seed!(1212)
A = gen_qr_problem_4(maxint=1)
A = A[:,1:3]//1
py_show("A=",A)

In [ ]:
%%julia
AtAinv = A'A \ 1I(3)//1
py_show( L"P = A \left(A^T A \right)^{-1} A^T = ", A*AtAinv*A')

In [ ]:
%%julia
W = gram_schmidt_w(Int.(A))
Q = normalize_columns( W )

py_show( L"\text{ Given } A = QR,\; P = Q Q^T =", Q*Q')

In [ ]:
%%julia
Uᵣ = svd(A).U
py_show( L"\text{ Given } A = U_r \Sigma_r V_r^T, \; P = U_r U_r^T =", 1//361, (x->Int.(round(N(x)))).(361*Uᵣ*Uᵣ'))

# 2. From Vectors to Vectors and Subspaces: Defining Angles

We will proceed gradually in this section, first discussing angles between vectors, vectors and lines,<br>
and ending with angles betweentwo lines (one dimensional subspaces).

## 2.1 Angle Between Two Vectors

The dot product and Cauchy's inequality resulted in the **definition of the angle $\mathbf{\theta}$** between two non-zero vectors $u$ and $v$:

$\qquad \cos \theta = \frac{ u \cdot v}{ \Vert u \Vert \ \Vert v \Vert}$.

**Intuition:**<br>
- Given two vectors $u$ and $v$, their **dot product** $u \cdot v$ measures **how aligned** the two vectors are.
- If $u$ and $v$ point in exactly the same direction, the angle $\theta$ between the vectors is given by $\theta = 0^\circ$ and $\cos\theta = 1$.
- If $u$ and $v$ are orthogonal (perpendicular), the angle $\theta = 90^\circ$ and $\cos\theta = 0$.

Thus, $\ \cos\theta\ $ is a **normalized measure of alignment** between the two vectors!<br>
$\qquad$ **In many applications we simply use $\mathbf{\cos \theta}$ directly, instead of computing $\mathbf{\theta}$.**

____
**Remarks:**
* Since a 1D subspace is just the set of all scalar multiples of a nonzero vector,<br>
the **angle between a vector and a line** is simply the angle between the vector and a basis vector for this 1D subspace.
* This generalizes to the angle between two lines, i.e., the angle between respective basis vectors.

This is shown in the following visualization.

In [ ]:
VectorAngleViewer().servable()

**Remark:**<br>
$\qquad$ Even though the demo draws 2D vectors, this idea generalizes to vectors in $\mathbb{R}^n$ for $n \ge 2$,<br>
$\qquad$ and the subspaces (lines) spanned by these vectors.

## 2.2 Angle between a Vector and a Subspace

We next generalize the angle between a vector and a line (a 1D subspace) to the angle between a line and a $k$-dimensional subspace.

<div style="background-color:#F2F5A9;color:black;padding-bottom:0.05cm;">

**Def:** Let $v \in \mathbb{R}^n$ be a nonzero vector, and $S \subseteq \mathbb{R}^n$ be a subspace.

$\qquad$ The **angle** $\theta$ between $v$ and $S$ is the unique angle $\theta \in [0^\circ, 90^\circ]$ between $v$ and its orthogonal projection onto $S$.
</div>

Formally:

$\qquad \cos\theta = \frac{ \Vert P v\Vert }{ \Vert v\Vert } \quad \Rightarrow \quad
\theta = \cos^{-1}\left( \frac{ \Vert P v\Vert }{ \Vert v\Vert } \right)
$
- Here, $P v$ is the orthogonal projection of $v$ onto $S$.
- $\Vert P v \Vert$ is the length of the projection.
- $\Vert v \Vert$ is the original length of $v$.



____
<div style="float:left;padding-right:1cm;">

**Intuition:**

- If $v$ lies **inside** $S$, then the projection $P v = v$, so $\theta = 0^\circ$.
- If $v$ is **orthogonal** to $S$, then $P v = 0$, so $\theta = 90^\circ$.
- In general, $0^\circ \le \theta \le 90^\circ$.

Geometrically, $\theta$ measures<br> **how much $v$ "leans toward"** the subspace $S$.

</div>
<div style="float:left;padding-left:1cm;border-left:2px solid black;">

**Geometric Interpretation:**

The angle $\theta$ corresponds to the angle between
- the vector $v$, and
- its projection $P v$.

In other words, $\theta$ is the **smallest angle between $v$ and any vector in $S$**.

Think of it as the **"tilt" of $v$ relative to the subspace.**
</div>

**Interactive Visualization:**

Use the demo below to explore how the angle changes as the direction of $v$ moves

In [ ]:
VectorProjectionDemo().servable()

____
**Remark:** The following lemma is important since it characterizes the angle in a way that will generalize:<br>
$\qquad$ if we measure the angle between the vector and any vector in the subspace that is not<br>
$\qquad$ on the line from the origin to $P v$, it is larger!

<div style="background-color:#F2F5A9;color:black;padding-bottom:0.05cm;">

**Lemma:** Let: $v \in \mathbb{R}^n$ be a nonzero vector, $S \subseteq \mathbb{R}^n$ be a subspace,<br>
$\qquad$ and $P v$ be the orthogonal projection of $v$ onto $S$.

Then among all nonzero vectors $s \in S$,
the vector $P v\;$ (and any nonzero scalar multiple of it)<br>
$\qquad$ forms the **smallest angle** with $v$, i.e.,

$\qquad
\theta(P v, v) \leq \theta(s, v) \quad \text{for all } s \in S, \; s \neq 0,\quad
$ with equality if and only if $s$ is a scalar multiple of $P v$.
</div>

**Proof Sketch:**
- The cosine of the angle between $v$ and $s$ is $\cos \theta = \frac{v \cdot s}{\|v\| \|s\|}$.
- This quantity is maximized (hence angle minimized) when $s$ points along $P v$.
- Since $P v$ is the closest vector to $v$ lying in $S$, any deviation from it decreases the cosine and thus increases the angle.

Thus, the direction of the orthogonal projection is geometrically the **best match to the original vector** within the subspace.

# 3. Angles Between Subspaces

**Objective:**<br>
$\qquad$ We now extend the idea of the angle between a vector and a subspace to define **angles between two subspaces**.<br>
$\qquad$ These are called the **principal angles**.

## 3.1 Principal Angles: Conceptual Introduction

Given two subspaces $\mathcal{U}, \mathcal{V} \subseteq \mathbb{R}^n$, the **principal angles** between them measure how "aligned" the two subspaces are.

The **sequence of principal angles** $\theta_1, \theta_2, \dotsc, \theta_d$ (where $d = \min(\dim(\mathcal{U}), \dim(\mathcal{V}))$) is defined recursively as follows
<div style="float:left;padding-right:1cm;">

- $\theta_1$ is the smallest angle between any nonzero vectors $u_1 \in \mathcal{U}$ and $v_1 \in \mathcal{V}$,
- subject to $\|u_1\| = \|v_1\| = 1$,
- $\theta_2$ is the smallest angle between directions orthogonal to $u_1$ and $v_1$,
- and so on.

Thus:

- $\theta_1$ captures the **best alignment**,
- $\theta_2$ captures the **best alignment in directions orthogonal to the first**,
- and so on.
</div>
<div style="float:left;padding-left:1cm;border-left:2px solid black;">

**Example:** consider **two planes** that intersect along a line.<br>
* $\theta_1$ measures the smallest angle between any two non-zero vectors, one from each plane.<br>
Along the intersection line, the vectors align perfectly, giving $\theta_1 = 0$.
* $\theta_2$ measures the angle between directions orthogonal to the intersection,<br>
capturing the **inclination** between the planes.

Thus:
- $\theta_1$ reflects the shared intersection,
- $\theta_2$ captures the relative tilt between the planes.
</div>

In [ ]:
PrincipalAnglePlayer().servable()

<div style="float:left;padding-left:0.5cm;">

| Setting | Principal Angles |
|:---|:---|
| Two nonzero vectors $u, v$ | Single angle between them |
| A vector $v$ and a subspace $S$ | Angle between $v$ and its projection onto $S$ |
| Two subspaces $U, V$ | Sequence of angles describing all relative alignments |
</div>

## 3.2 Principal Angles Via Projections

To compute principal angles between subspaces $\mathcal{U}$ and $\mathcal{V}$,<br>
we study how **orthogonal projections** onto these subspaces interact.

Let $\mathcal{U}, \mathcal{V} \subseteq \mathbb{R}^n$ be subspaces of dimensions $k$ and $\ell$.<br>
$\qquad$ Let $U \in \mathbb{R}^{n \times k}$ and $V \in \mathbb{R}^{n \times \ell}$<br>
$\qquad$ have orthonormal columns spanning $\mathcal{U}$ and $\mathcal{V}$, respectively, i.e, $\;U^T U = I_k, \quad V^T V = I_\ell.$<br>
$\qquad$ The corresponding orthogonal projections are $\; P_u = U U^T, \quad P_v = V V^T$.

Given any vector $x \in \mathbb{R}^n$
- Project $x$ onto $\mathcal{V}$ to obtain $P_v x$,
- Then project $P_v x$ onto $\mathcal{U}$, yielding $P_u P_v x$.

As $x$ varies, $P_v x$ sweeps out all of $\mathcal{V}$.<br>
Thus, the **angles between $\mathbf{P_v x}$ and $\mathbf{P_u P_v x}$** capture the alignment between $\mathcal{U}$ and $\mathcal{V}$.

This interaction is encoded by the matrix product $\;\mathbf{P_u P_v}\;$, which we now explore in detail.

### 3.2.1 Properties of the Projection Product $P_u P_v$

We now study the product $P_u P_v$, which plays a central role in understanding principal angles.

**Key observations:**
- $P_u$ and $P_v$ are **symmetric** and **idempotent**<br>
  $
  P_u = P_u^T, \quad P_u^2 = P_u, \quad P_v = P_v^T, \quad P_v^2 = P_v
  $
- However, their product $P_u P_v$ is generally **neither symmetric** nor a projection<br>
  $
  (P_u P_v)^T = P_v P_u, \quad (P_u P_v)^2 \neq P_u P_v \quad \text{in general.}
  $
- Despite this, $P_u P_v$ encodes important geometric information about the relationship between $\mathcal{U}$ and $\mathcal{V}$.

**Important special cases:**
- If $\mathcal{U} = \mathcal{V}$, then $P_u = P_v$ and $P_u P_v = P_u$ is a projection.
- If $\mathcal{U} \perp \mathcal{V}$, then $P_u P_v = 0$.
- In general, the behavior of $P_u P_v$ reflects how much $\mathcal{V}$ "leans into" $\mathcal{U}$.

### 3.2.2 Example: a Simple 2D Case

To build intuition, consider two lines (1D subspaces) in $\mathbb{R}^2$.

Let $\;\mathcal{U}$ = x-axis, with orthonormal basis $U = \begin{pmatrix}1 \\ 0\end{pmatrix},\;$
and $\;\mathcal{V}$ = line at angle $\theta$, with orthonormal basis $V = \begin{pmatrix}\cos \theta \\ \sin \theta\end{pmatrix}$.

The projection matrices and their product are <br>$\qquad
P_u = U U^T = \begin{pmatrix}1 & 0 \\ 0 & 0\end{pmatrix}, \qquad
P_v = V V^T = \begin{pmatrix}\cos^2 \theta & \cos \theta \sin \theta \\ \cos \theta \sin \theta & \sin^2 \theta\end{pmatrix},\qquad
P_u P_v = \begin{pmatrix} \cos^2 \theta & \cos \theta \sin \theta \\ 0 & 0 \end{pmatrix}.
$

- $P_u P_v$ is **not symmetric**, nor a projection matrix.
- Its eigenvalues are $0$ and $\cos^2 \theta$.
- Thus, the singular values of $P_u P_v$ are $0$ and $\cos^2 \theta$.

This reflects the principal angle $\theta$ between $\mathcal{U}$ and $\mathcal{V}$.

### 3.2.3 Action of the Linear Transformation $P_u P_v$

Given an arbitrary vector $x \in \mathbb{R}^n$:
- First, $P_v x$ is the orthogonal projection of $x$ onto $\mathcal{V}$,
- Then, $P_u$ is applied to $P_v x$, producing $P_u P_v x$.

Thus, $P_u P_v$ maps vectors in $\mathbb{R}^n$ into $\mathcal{U}$, but it is not a projection operator in general.

---
**Properties of $P_u P_v$**
- $P_u P_v$ is a linear transformation.
- In general:
  - $P_u P_v$ is **not a projection**: $(P_u P_v)^2 \neq P_u P_v$ unless $\mathcal{U} = \mathcal{V}$,
  - $P_u P_v$ is **not symmetric**: $(P_u P_v)^T \neq P_u P_v$ unless $\mathcal{U} = \mathcal{V}$.
- Special cases:
    - If $\mathcal{U} = \mathcal{V}$, then $P_u P_v = P_u = P_v$ (an orthogonal projection).<br>
      Since $U = V$, we have $P_u P_v = U U^T V V^T = U U^T U U^T = U U^T$.
    - If $\mathcal{U} \perp \mathcal{V}$, then $P_u P_v = 0$.<br>
      Since $U^T V = 0$, we have $P_u P_v = U U^T V V^T = U (0) V^T = 0$.

### 3.2.4 The Symmetric Operator $P_v P_u P_v$

Since $P_u P_v$ is not symmetric in general, we instead study $\;P_v P_u P_v\;$
which acts entirely within the subspace $\mathcal{V}$.

This operator is symmetric and positive semidefinite, with eigenvalues lying in $[0,1]$.

---
**$\mathbf{P_v P_u P_v}$ is symmetric**

$\qquad
(P_v P_u P_v)^T = (V V^T U U^T V V^T)^T = V V^T U U^T V V^T = P_v P_u P_v.
$

---
**$\mathbf{P_v P_u P_v}$ is positive semidefinite**

For any $x \in \mathbb{R}^n$,<br>
$\qquad
x^T (P_v P_u P_v) x = (P_v x)^T P_u (P_v x) = (P_v x)^T (U U^T) (P_v x) = \Vert U^T (P_v x) \Vert^2 \geq 0.
$$

---
**The eigenvalues of $\mathbf{P_v P_u P_v}$ lie in $[0,1]$**

Let $(\lambda, x)$ be an eigenpair of $P_v P_u P_v$. Since $P_v P_u P_v$ maps into $\mathcal{V}$, we have $x \in \mathcal{V}$.

$\qquad\begin{align}
P_v P_u P_v x = \lambda x \quad
& \Rightarrow\quad P_v P_u x            = \lambda x \quad            &\text{(since } P_v x = x \text{)} \\
& \Rightarrow\quad \Vert P_v P_u x\Vert = \lambda \Vert x\Vert \quad &(\lambda \geq 0\ \text{since positive semidefinite}) \\
& \Rightarrow\quad \Vert P_u x\Vert \geq \lambda \Vert x \Vert \quad &(\text{since } P_v \text{ is a contraction})
\end{align}$

However, $\Vert P_u x \Vert \leq \Vert x \Vert$ because $P_u$ is an orthogonal projection.

Thus
$\;
0 \leq \lambda \leq 1.
$

---
**Geometric Interpretation**

Given $v \in \mathcal{V}$:
- $P_v v = v$,
- $P_u v$ projects $v$ orthogonally onto $\mathcal{U}$,
- $P_v P_u v$ projects $P_u v$ back into $\mathcal{V}$.

Thus, $P_v P_u P_v$ measures how much of a vector survives two successive projections: first into $\mathcal{U}$, then back into $\mathcal{V}$.

### 3.2.5 Principal Angles and the SVD of $U^T V$

We are ready for the main result of our analysis:

<div style="background-color:#F2F5A9;color:black;padding-bottom:0.05cm;padding-top:0.05cm;">

**Thm:** (Principal Angles via Singular Values and Projection Operators)
<div style="padding-left:0.8cm;">
Let $\mathcal{U}, \mathcal{V} \subseteq \mathbb{R}^n$ be subspaces with dimensions $k = \dim(\mathcal{U})$ and $\ell = \dim(\mathcal{V})$.<br>
Let $U \in \mathbb{R}^{n \times k}$ and $V \in \mathbb{R}^{n \times \ell}$ be matrices whose columns form orthonormal bases for $\mathcal{U}$ and $\mathcal{V}$, respectively.

Form the matrix of inner products:
$\qquad
M = U^T V \in \mathbb{R}^{k \times \ell}.
$

Then:
- The singular values $\sigma_1 \geq \sigma_2 \geq \dotsc \geq \sigma_d$ of $M$, where $d = \min(k,\ell)$, satisfy
$\;\;
0 \leq \sigma_i \leq 1.
$
- The **principal angles** $\theta_1, \dotsc, \theta_d$ between $\mathcal{U}$ and $\mathcal{V}$ are given by
$\;\;
\cos(\theta_i) = \sigma_i, \quad \text{for} \quad i = 1, \dotsc, d.
$
- Equivalently, the **squares** of the singular values, $\sigma_i^2$, are the nonzero eigenvalues $\lambda_i$ of the matrix $P_v P_u P_v$,<br>
where $P_u = U U^T$ and $P_v = VV^T$ are the orthogonal projections onto $\mathcal{U}$ and $\mathcal{V}$:
$\;\;
\cos^2(\theta_i) = \lambda_i.
$

- The principal vectors are given by $u_i = U a_i,\; v_i = V b_i, \; i=1,\dots d$
- If $\lambda_i = 0$, the corresponding principal angle is $90^\circ$.<br>
In this case, there is no associated principal vector $u_i$ aligned to $v_i$ (since $P_u v_i = 0$).
</div></div>

#### Proof

The proof requires several steps and starts with an analysis of the eigenproblem for $P_v P_u P_v$.

Let $(\lambda_i, v_i)$ be an eigenpair of $P_v P_u P_v$ with $\lambda_i > 0$ and $\Vert v_i \Vert = 1$,<br>
$\qquad$ and define $u_i$ as the normalized projection of $v_i$ onto $\mathcal{U}$ by
$\;
u_i = \frac{P_u v_i}{\Vert P_u v_i \Vert}.
$

$\qquad$ Thus $\;v_i \in \mathcal{V},\;$, and $\;u_i \in \mathcal{U}$.

We seek to compute the angle between $u_i$ and $v_i$.

---

##### Step 1: Compute the Angle Between $u_i$ and $v_i$

Since $P_v P_u P_v$ maps into $\mathcal{V}$,
$\;
P_v P_u P_v v_i = \lambda_i v_i \quad \Rightarrow \quad v_i \in \mathcal{V} \quad \Rightarrow \quad P_v v_i = v_i.
$

From the eigenpair equation
$\;
\lambda_i v_i = P_v P_u P_v v_i = P_v P_u v_i,
$<br>
$\qquad$ thus $P_u v_i \neq 0$ when $\lambda_i \neq 0$ and $u_i$ is well defined.

We now compute the norm of $P_u v_i$<br>
$\qquad\begin{align}
\Vert P_u v_i \Vert^2
&= v_i^T P_u v_i \qquad\qquad\qquad &(\text{projection property: } P_u^2 = P_u) \\
&= (P_v v_i)^T P_u v_i              &(\text{since } P_v v_i = v_i) \\
&= v_i^T P_v P_u v_i                &(\text{since } P_v^T = P_v) \\
&= v_i^T P_v P_u P_v v_i            &(\text{again since } P_v = P_v^T) \\
&= \lambda_i v_i^T v_i              &(\text{from the eigenproblem}) \\
&= \lambda_i                        &(\text{since } \Vert v_i \Vert = 1).
\end{align}$

Thus
$\;
\Vert P_u v_i \Vert = \sqrt{\lambda_i}.
$

---

We next compute the cosine of the angle $\theta_i$ between $u_i$ and $v_i$:<br>
$\qquad\begin{align}
\cos \theta_i
&= \langle u_i, v_i \rangle \quad &(\text{since both are unit vectors}) \\
&= \frac{\langle P_u v_i, v_i \rangle}{\Vert P_u v_i \Vert} \quad &(\text{definition of } u_i) \\
&= \frac{v_i^T P_u v_i}{\Vert P_u v_i \Vert} \quad &(\text{matrix form}) \\
&= \frac{(P_v v_i)^T P_u v_i}{\Vert P_u v_i \Vert} \quad &(\text{since } P_v v_i = v_i) \\
&= \frac{v_i^T P_v P_u v_i}{\Vert P_u v_i \Vert} \quad &(\text{symmetry of } P_v) \\
&= \frac{\lambda_i v_i^T v_i}{\Vert P_u v_i \Vert} \quad &(\text{eigenproblem again}) \\
&= \frac{\lambda_i}{\sqrt{\lambda_i}} \\
&= \sqrt{\lambda_i}.
\end{align}$

Thus
$\;\;
\cos^2 \theta_i = \lambda_i.
$

---

#### Step 2: Relation to the Singular Values of $M = U^T V$

Expand $P_v P_u P_v = V (V^T U)(U^T V) V^T = V (M^T M) V^T,\;
$
where
$\;
M = U^T V \in \mathbb{R}^{k \times \ell}.
$

$\qquad$ Note that $M$ encodes the inner products between orthonormal bases of $\mathcal{U}$ and $\mathcal{V}$.

Now consider the singular value decomposition (SVD) of $\;M = A \Sigma B^T,\;$
where $\Sigma = \operatorname{diag}(\sigma_1, \dotsc, \sigma_d)$, $d = \min(k, \ell)$.

$\qquad$ Thus
$\;
M^T M = B \Sigma^T \Sigma B^T,\;
$
and the eigenvalues of $M^T M$ are $\sigma_i^2$.

$\qquad$ Since
$\;
P_v P_u P_v = V (M^T M) V^T,\;
$
and $V$ has orthonormal columns, the nonzero eigenvalues $\lambda_i$ of $P_v P_u P_v$ are the eigenvalues of $M^T M$.

Hence
$\;
\lambda_i = \sigma_i^2,\;
$
and recalling that
$\;
\cos^2 \theta_i = \lambda_i,
\;$
we conclude
$\;
\cos \theta_i = \sigma_i.
$

---

#### Step 3: Conclusion

Finally, by standard properties of the singular value decomposition (SVD),
the singular values $\sigma_i$ correspond to the maximum cosines between directions in $\mathcal{U}$ and $\mathcal{V}$
under orthogonality constraints on previous choices.

Thus:
- $\theta_i$ are the principal angles between $\mathcal{U}$ and $\mathcal{V}$,
- $u_i$ and $v_i$ are the corresponding principal vectors.

### 3.2.6 Symmetry of Principal Angles

<div style="background-color:#F2F5A9;color:black;padding-top:0.2cm;padding-bottom:0.2cm;">

**Thm:** (Principal Angles Are Symmetric)

<div style="padding-left:0.8cm;">
Let $\mathcal{U}, \mathcal{V} \subseteq \mathbb{R}^n$ be subspaces with orthonormal bases $U \in \mathbb{R}^{n \times k}$ and $V \in \mathbb{R}^{n \times \ell}$.

Form the matrix of inner products $M = U^T V \in \mathbb{R}^{k \times \ell}$.

Then:
- The singular values of $M$ and $M^T$ are identical.
- Therefore, the principal angles between $\mathcal{U}$ and $\mathcal{V}$ are the same as the principal angles between $\mathcal{V}$ and $\mathcal{U}$.

</div>
</div>

The singular values of a matrix $M$ are the square roots of the eigenvalues of $M^T M$ and $M M^T$.

Since $M \in \mathbb{R}^{k \times \ell}$:

- $M^T M$ is $\ell \times \ell$
- $M M^T$ is $k \times k$

Moreover, $M^T M$ and $M M^T$ share the same nonzero eigenvalues.
Thus, $M$ and $M^T$ have the same nonzero singular values.

$\qquad\therefore\quad$ The singular values of $U^T V$ and $V^T U$ are the same.

Since the principal angles are determined by these singular values, we conclude:

$\qquad\therefore\quad$ The principal angles between $\mathcal{U}$ and $\mathcal{V}$ are the same as between $\mathcal{V}$ and $\mathcal{U}$.

## 3.3 Computation of Principal Angles and Principal Vectors

In practice, the principal angles and vectors between two subspaces can be computed by the following simple procedure:

<div style="float:left; padding-left:1cm;">

**Step-by-Step Summary**

| Step | Description |
|:---|:---|
| 1 | Choose orthonormal bases $U$ for $\mathcal{U}$ and $V$ for $\mathcal{V}$ |
| 2 | Form the matrix of inner products $M = U^T V$ |
| 3 | Compute the singular value decomposition (SVD) $M = A \Sigma B^T$ |
| 4 | Principal angles: $\cos(\theta_i) = \sigma_i$ |
| 5 | Principal vectors: $u_i = U a_i,\; v_i = V b_i$ |
</div>

The singular values $\sigma_i$ encode the cosines of the principal angles $\theta_i$, and the left and right singular vectors $(a_i, b_i)$ specify the principal directions within the subspaces.

Thus, the geometry of the relationship between $\mathcal{U}$ and $\mathcal{V}$ is completely determined by the SVD of $U^T V$.

### 3.3.1 Example: Subspaces with a Shared and an Orthogonal Direction

Consider two subspaces $\mathcal{U}, \mathcal{V} \subseteq \mathbb{R}^4$, each of dimension 2:

- $\mathcal{U}$ is spanned by orthonormal vectors $u_1, u_2$,
- $\mathcal{V}$ is spanned by $v_1, v_2$:
  - $v_1$ aligned with $u_1$ (perfect overlap),
  - $v_2$ orthogonal to both $u_1$ and $u_2$ (no overlap).

Thus, the expected principal angles are $\theta_1 = 0^\circ$, $\theta_2 = 90^\circ$.

In [109]:
%%julia
# ---------------------------------------------------------------------------------------------------
# Step 1: obtain orthonormal bases
U = [1 0; 0 1; 0 0; 0 0]  # Define orthonormal basis for U
V = [1 0; 0 0; 0 0;  0 1]  # Define orthonormal basis for V
# ---------------------------------------------------------------------------------------------------
# Step 2: Matrix of inner products
M = U'V
# ---------------------------------------------------------------------------------------------------
# Step 3: Perform SVD
S = svd(M)
A, Σ, B = S.U, S.S, S.V

# Step 4: Principal angles
θ = Int.(acosd.(Σ))
A = Int.(A); Σ=Int.(Σ); B = Int.(B)
# ---------------------------------------------------------------------------------------------------
# Step 5: Principal vectors
u₁ = U * A[:, 1]
u₂ = U * A[:, 2]

v₁ = V * B[:, 1]
v₂ = V * B[:, 2]

# ---------------------------------------------------------------------------------------------------
# Display results
display(py_show( "Step 1:",L"\quad", "Basis matrices ", L"\quad U =", U, L",\quad V = ", V))
display(py_show( "Step 2:",L"\quad", "Matrix of inner products M = Uᵀ V:", L"\quad M =", M ))
display(py_show( "Step 3:",L"\quad", "SVD(M) = ", A, Diagonal(Σ), B))
display(py_show( "Step 4:",L"\quad", "Principal angles (in degrees):", θ))

display(py_show( "Steps 5:",L"\quad", "Principal vectors"))
display(py_show( L"\qquad", "1st principal vector pair (u₁, v₁) = ", u₁, v₁, L",\quad", "angle ", θ[1]))
display(py_show( L"\qquad", "2nd principal vector pair (u₂, v₂) = ", u₂, v₂, L",\quad", "angle ", θ[2]))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

**Interpretation:**
- The first principal vectors $u_1$ and $v_1$ are perfectly aligned, corresponding to a principal angle of $0^\circ$.
- The second principal vectors $u_2$ and $v_2$ are fully orthogonal, corresponding to a principal angle of $90^\circ$.
- This confirms the expected geometric relationship between $\mathcal{U}$ and $\mathcal{V}$.


### 3.3.2 Example: Partial Overlap with Zero Singular Values

We define two subspaces:

- $\mathcal{U}$ of dimension 2,
- $\mathcal{V}$ of dimension 3,
- both in $\mathbb{R}^4$.

In [141]:
%%julia
# ---------------------------------------------------------------------------------------------------
# Step 1: obtain orthonormal bases
U = [1 0; 0 1; 0 0;0 0]

v1 = [c; s; 0; 0]   # tilted in (e₁,e₂) plane
v2 = [0.1; 0.1; 0.98; 0.0]   # slight tilt toward (e₁,e₂) plane
v2 = v2 / norm(v2)           # normalize
v3 = [0; 0; 0; 1]            # fully in e₄ direction
V_raw = hcat(v1, v2, v3)
Q, R = qr(V_raw)   # Thin QR by default
V = Matrix(Q)      # Extract Q explicitly as a dense matrix# ---------------------------------------------------------------------------------------------------

# ---------------------------------------------------------------------------------------------------
# Step 2: Matrix of inner products
M = 1.0U'V
# ---------------------------------------------------------------------------------------------------
# Step 3: Perform SVD
S = svd(M, full=false)
A, Σ, B = S.U, S.S, S.V

# ---------------------------------------------------------------------------------------------------
# Step 4: Principal angles
θ = acosd.(clamp.(Σ, 0, 1))
# ---------------------------------------------------------------------------------------------------
# Step 5: Principal vectors
u₁ = U * A[:, 1]
v₁ = V * B[:, 1]

u₂ = U * A[:, 2]
v₂ = V * B[:, 2]
# ---------------------------------------------------------------------------------------------------
# Display results
display(py_show( "Step 1:",L"\quad", "Basis matrices ", L"\quad U =", U, L",\quad V = ", (V, number_formatter=x->round_value(x,2))))
display(py_show( "Step 2:",L"\quad", "Matrix of inner products M = Uᵀ V:", L"\quad M =",( M, number_formatter=x->round_value(x,2))))
display(py_show( "Step 3:",L"\quad", "SVD(M) = ", set(A, Diagonal(Σ), B, number_formatter=x->round_value(x,2), setstyle=:array, separator="")))
display(py_show( "Step 4:",L"\quad", "Principal angles (in degrees):", θ, number_formatter=x->round_value(x,2)))

display(py_show( "Steps 5:",L"\quad", "Principal vectors"))
display(py_show( L"\qquad", "1st principal vector pair (u₁, v₁) = ", u₁, v₁, L",\quad", "angle ", θ[1], number_formatter=x->round_value(x,2)))
display(py_show( L"\qquad", "2nd principal vector pair (u₂, v₂) = ", u₂, v₂, L",\quad", "angle ", θ[2], number_formatter=x->round_value(x,2)))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

**Interpretation:**
- The first principal vector pair ($u₁, v₁$) are collinear along the line of intersection of the planes $\theta_1 = 0^\circ$ relative to each other.
- The second principal vectors ($u₂, v₂$) are close to orthogonal $\theta_2.

# 4. Take Away

**Main Points:**

* Principal angles and vectors quantify the **relative orientation** between two subspaces $\mathcal{U}$ and $\mathcal{V}$.
* The subspaces can have **different dimensions**.<br>
There are $d = \min\big( \dim(\mathcal{U}),\ \dim(\mathcal{V}) \big)$ principal angles $\theta_1, \dotsc, \theta_d$.
* Principal angles are computed from the **SVD** of the matrix $M = U^T V$,<br>
where the **columns** of $U$ and $V$ are orthonormal bases for $\mathcal{U}$ and $\mathcal{V}$.
* The singular values $\sigma_i$ of $M$ relate to the principal angles by $\quad \cos(\theta_i) = \sigma_i.$
* The associated **principal vectors** are
  * $u_i = U a_i$ in $\mathcal{U}$,
  * $v_i = V b_i$ in $\mathcal{V}$,
where $a_i, b_i$ are the singular vectors of $M$.

---
**Special Cases:**
* $\theta_i = 0^\circ$ : the subspaces **overlap perfectly** along some direction.
* $\theta_i = 90^\circ$ : the subspaces are **orthogonal** along that mode.

---
**Summary:**<br>
$\quad$ Principal angles provide a **precise** and **computationally robust** way to measure how two subspaces "tilt" relative to each other.

## X.x Eigenvalues of Products of Orthogonal Projection Matrices

We will prove that **all eigenvalues $\lambda$ of $P_u P_v$ satisfy $0 \leq \lambda \leq 1$**.

Note that $P_u P_v$ is not symmetric in general, but we can study the related symmetric matrix:

$ \qquad
P_v P_u P_v = (P_v P_u)(P_v P_u)^T = V V^T U U^T V V^T
$

<div style="background-color:#F2F5A9;color:black;padding-bottom:0.05cm;">

**Thm:** Given orthogonal projection matrices $P_u$ and $P_v$. Then<br>
$\qquad P_u P_v$ is **positive semi-definite** with eigenvalues $0 \le \lambda \le 1$.
</div>

Let $(\lambda, x)$ be an eigenpair of $P_u P_v$ and note that if $P_v x = 0$ we have $\lambda=0$.

For $P_v x \ne 0$, we have<br>
$\quad \begin{align}
P_u P_v x = \lambda x
\quad & \Rightarrow \quad   x^T P_v P_u P_v x &=&\ \lambda\  x^T P_v x \\
& \Rightarrow \quad x^T (P_u P_v)^T P_u P_v x &=&\ \lambda\ x^T P^T_v P_v x \\
& \Rightarrow \quad \Vert P_u P_v x \Vert^2 &=&\ \lambda\ \Vert P_v x \Vert^2 \\
& \Rightarrow \quad \lambda  &=&\ \frac{\Vert P_u P_v x \Vert^2}{\Vert P_v x \Vert^2} \\
& \Rightarrow \quad \lambda  &\le&\ \frac{\Vert P_v x \Vert^2}{\Vert P_v x \Vert^2} = 1 \\
\end{align}$

where we have used $\Vert P_u y \Vert \le \Vert y \Vert$ for any $y$ since $P_u$ is a projection operator.

## X.y Interpretation of the Eigenvalues of $P_u P_v$

### 3.3.1 Geometric Meaning of Eigenvalues

Let $(\lambda, x)$ be an eigenpair of $P_u P_v$, i.e., $\;\;P_u P_v x = \lambda x$

$\qquad$ Thus $\lambda$ describes how much of $x$ survives when first projected onto $\mathcal{V}$ and then onto $\mathcal{U}$.

#### Case: Eigenvalue $\lambda = 1$

If $\lambda = 1$, then $\;P_u P_v x = x$

Therefore
- $P_v x \in \mathcal{U}$
- $x \in \mathcal{V}$ (since $P_v$ doesn't change $x$)
- $x \in \mathcal{U}$ (since $P_u$ doesn't change it either)

$\therefore$ **Eigenvalue 1 corresponds to directions in the intersection** $\mathcal{U} \cap \mathcal{V}$.<br>
$\qquad$ If $\dim(\mathcal{U} \cap \mathcal{V}) = d$, then $P_u P_v$ has **$\mathbf{d}$ eigenvalues equal to 1**.

#### Case: Eigenvalue $\lambda = 0$

If $\lambda = 0$, then $\;P_u P_v x = 0$

This can happen in several ways
1. **$x \in \mathcal{V}^\perp$**: so $P_v x = 0$
2. **$P_v x \in \mathcal{U}^\perp$**: so $P_u P_v x = 0$ even though $P_v x \ne 0$
3. **$x \in \mathcal{V}$ but orthogonal to $\mathcal{U}$**

In all cases, this means<br>
$\qquad$ **$x$ is annihilated by the chain of projections.**<br>
$\qquad$ That is, nothing survives both projections.

$\therefore$ **Eigenvalue 0 corresponds to directions in** $\mathcal{V}$ **that are orthogonal to** $\mathcal{U}$
**or vice versa.**

---

#### Summary: Eigenvalue Interpretation

<div style="width:50%;">

| Eigenvalue $\lambda$ | Meaning |
|----------------------|---------|
| $\lambda = 1$        | $x \in \mathcal{U} \cap \mathcal{V}$ (shared direction) |
| $\lambda = 0$        | either $x$ lies in $\mathcal{V}$ but orthogonal to $\mathcal{U}$, or $P_v x = 0$ |
| $0 < \lambda < 1$    | Partial overlap — $x$ projects nontrivially into both |
</div>

**Remark:**<br>
$\qquad$ The **multiplicity of $\lambda = 1$** equals $\dim(\mathcal{U} \cap \mathcal{V})$ <strong style="color:red;">

## 3.4 $P_u P_v$ and $P_v P_u$

We now examine the **composition** of $P_u$ and $P_v$

$\qquad
P_u P_v \ne P_v P_u\;\;
$ in general

I.e., **projection matrices do not commute**. The order of projection **matters**.

- $P_u P_v x$ means:
  first project $x$ onto $\mathcal{V}$, then project that result onto $\mathcal{U}$.
- $P_v P_u x$ means:
  first project $x$ onto $\mathcal{U}$, then project that result onto $\mathcal{V}$.

Unless one subspace is contained in the other, or they are aligned, the results will differ.

#### Shared Properties

Despite not being equal, they share some important features:

- Both $P_u P_v$ and $P_v P_u$ are **real** and **diagonalizable**
- They have the **same nonzero eigenvalues**
- Their eigenvalues lie in $[0, 1]$
- The non-zero eigenvalues If $\theta_1, \dots, \theta_k$ define **principal angles** between $\mathcal{U}$ and $\mathcal{V}$, i.e.,

$\qquad
\text{Nonzero eigenvalues of } P_u P_v \text{ and } P_v P_u \text{ are } \cos^2(\theta_i)
$

This arises because they are related via SVD of $U^T V$.

#### Algebraic Explanation

Let $\;\;
U^T V = W_1 \Sigma W_2^T \quad \text{(SVD)}
$

Then
- $P_u P_v = U W_1 \Sigma W_2^T V^T$
- $P_v P_u = V W_2 \Sigma W_1^T U^T$

So they are **transposes** of each other!

$\qquad
(P_u P_v)^T = P_v P_u
$

Thus, they are **adjoint to one another**, and their **singular values** (and hence nonzero eigenvalues) are the same.

#### Key Difference: Eigenvectors

Even though $P_u P_v$ and $P_v P_u$ share eigenvalues, they typically have **different eigenvectors**.

- Eigenvectors of $P_u P_v$ lie in $\mathbb{R}^n$ and are generally **not preserved** under switching the order
- Only in special cases (e.g., subspaces aligned, or 1D) do they commute

#### Special Case: When Do They Commute?

- If $\mathcal{U} \subseteq \mathcal{V}$ (or vice versa)
- If $\mathcal{U} = \mathcal{V}$
- If both projections are onto the same line or plane

Then:

$\qquad
P_u P_v = P_v P_u = P_u = P_v
$

### Example: Comparing $P_u P_v$ and $P_v P_u$ in $\mathbb{R}^2$

Let’s work with two 1D subspaces in $\; \mathbb{R}^2 \;$:

- Let $\; \mathcal{U} \;$ be the $x$-axis:<br>
  $\qquad
  u = \begin{bmatrix} 1 \\ 0 \end{bmatrix}
  \Rightarrow
  P_u = uu^T =
  \begin{bmatrix}
  1 & 0 \\
  0 & 0
  \end{bmatrix}
  $
- Let $\; \mathcal{V} \;$ be a line at angle $\; \theta \;$ from the $x$-axis:<br>
  $\qquad
  v = \begin{bmatrix} \cos\theta \\ \sin\theta \end{bmatrix}
  \Rightarrow
  P_v = vv^T =
  \begin{bmatrix}
  \cos^2\theta & \cos\theta \sin\theta \\
  \cos\theta \sin\theta & \sin^2\theta
  \end{bmatrix}
  $

1. **Compute** $\; P_u P_v \;$:

$\qquad
P_u P_v =
\begin{bmatrix}
1 & 0 \\
0 & 0
\end{bmatrix}
\begin{bmatrix}
\cos^2\theta & \cos\theta \sin\theta \\
\cos\theta \sin\theta & \sin^2\theta
\end{bmatrix}
=
\begin{bmatrix}
\cos^2\theta & \cos\theta \sin\theta \\
0 & 0
\end{bmatrix}
$

2. **Now compute** $\; P_v P_u \;$:

$\qquad
P_v P_u =
\begin{bmatrix}
\cos^2\theta & \cos\theta \sin\theta \\
\cos\theta \sin\theta & \sin^2\theta
\end{bmatrix}
\begin{bmatrix}
1 & 0 \\
0 & 0
\end{bmatrix}
=
\begin{bmatrix}
\cos^2\theta & 0 \\
\cos\theta \sin\theta & 0
\end{bmatrix}
$

We clearly see:

$\qquad
P_u P_v \ne P_v P_u
$

Each matrix projects into a **different subspace** after applying the other. The **resulting matrix** depends on the order.

#### But: Same Nonzero Eigenvalue

Let’s compute their eigenvalues.

Both have characteristic polynomial:

$\qquad
\det(A - \lambda I) = \lambda(\lambda - \cos^2\theta)
$

So the eigenvalues of both are:

$\qquad
\boxed{\{0, \cos^2\theta\}}
$

These correspond to:
- A **zero mode**: directions orthogonal to both
- A **principal direction**: aligned with both subspaces

### Conclusion

This example shows:

- **Order matters** for composition of projections
- **But geometry still rules**: the amount of alignment (via principal angle) is shared

<div style="float:left;margin-left:1cm;">

| Property              | $P_u P_v$ and $P_v P_u$ |
|-----------------------|-------------------------|
| Equal?                | No, not in general      |
| Share eigenvalues?    | Yes                     |
| Eigenvalue range      | $[0, 1]$                |
| Eigenvectors?         | Generally different     |
| Transpose of each other? | Yes: $(P_u P_v)^T = P_v P_u$ |
| Commute when?         | If subspaces align or contain one another |

</div>

## 1.6 Principal Angles

### 🔷 Principal Angles and the Matrix $P_u P_v$

Let $\mathcal{U}, \mathcal{V} \subseteq \mathbb{R}^n$ be two subspaces of dimensions $k$ and $\ell$, respectively.

The **principal angles** $\theta_1, \theta_2, \dots, \theta_d$ between the subspaces are defined recursively as the **smallest angles between pairs of unit vectors**:

- $u_i \in \mathcal{U}$, $v_i \in \mathcal{V}$
- such that:
  - $\langle u_i, v_i \rangle = \cos(\theta_i)$
  - $u_i^T u_j = v_i^T v_j = 0$ for all $i \ne j$

Here, $d = \min(k, \ell)$ is the number of principal angles.

### 🎯 How They Arise from $P_u P_v$

Let:

- $P_u = UU^T$, where $U \in \mathbb{R}^{n \times k}$ is orthonormal basis for $\mathcal{U}$
- $P_v = VV^T$, with $V \in \mathbb{R}^{n \times \ell}$ for $\mathcal{V}$

Then:
- The matrix $U^T V \in \mathbb{R}^{k \times \ell}$ captures the **pairwise dot products** between basis vectors of the subspaces.
- Perform **SVD** on this matrix:<br>
  $\qquad
  U^T V = W_1 \Sigma W_2^T
  $
- The singular values $\sigma_i = \cos(\theta_i)$ are the **cosines of the principal angles**

### 🧠 Connection to $P_u P_v$

#### 🔍 Why Are the Eigenvalues of $P_U P_V$ the Squares of the Singular Values of $U^\top V$?

Let $\mathcal{U}, \mathcal{V} \subseteq \mathbb{R}^n$ be subspaces of dimensions $k$ and $\ell$, with orthonormal bases $U \in \mathbb{R}^{n \times k}$ and $V \in \mathbb{R}^{n \times \ell}$. Let:

- $P_U = UU^\top$ — projection onto $\mathcal{U}$
- $P_V = VV^\top$ — projection onto $\mathcal{V}$
- $M = U^\top V \in \mathbb{R}^{k \times \ell}$

---

#### Step 1: SVD of $U^\top V$

Compute the singular value decomposition (SVD):

$$
U^\top V = W_1 \Sigma W_2^\top
$$

Then:

- $\Sigma = \operatorname{diag}(\sigma_1, \dots, \sigma_d)$
- $\sigma_i = \cos(\theta_i)$ are the **cosines of the principal angles** between $\mathcal{U}$ and $\mathcal{V}$
- Define $A = UW_1$, $B = VW_2$, then:

$$
U^\top V = W_1 \Sigma W_2^\top \quad \Rightarrow \quad P_U P_V = A \Sigma B^\top
$$

---

#### Step 2: Action of $P_U P_V$

Let $z_i = B e_i = V W_2 e_i$. Then:

$$
P_U P_V z_i = A \Sigma e_i = \sigma_i A e_i
$$

Now apply $P_U P_V$ again:

$$
P_U P_V (A e_i) = A \Sigma^2 e_i = \sigma_i^2 A e_i
$$

So:
- $A e_i = U W_1 e_i$ is an eigenvector of $P_U P_V$
- With eigenvalue $\lambda_i = \sigma_i^2$

---

#### ✅ Conclusion

$$
\boxed{
\text{Eigenvalues of } P_U P_V \text{ are } \sigma_i^2 = \cos^2(\theta_i)
}
$$

That is:
- The **singular values** of $U^\top V$ are $\sigma_i = \cos(\theta_i)$
- The **eigenvalues** of $P_U P_V$ are the **squares** of these singular values

This directly links the **geometry of subspace alignment** (principal angles) to the **algebra of projection products**.


The matrix:

$\qquad
P_u P_v = U U^T V V^T
$

has **nonzero eigenvalues** equal to:

$\qquad
\cos^2(\theta_1), \cos^2(\theta_2), \dots, \cos^2(\theta_d)
$

So:

> **Principal angles govern the nonzero eigenvalues of the matrix product $P_u P_v$**

This reflects how "aligned" the subspaces are in each direction.

### 🧪 Summary of Geometric Meaning

<div style="width:50%;">

| Principal Angle $\theta_i$ | Interpretation |
|----------------------------|----------------|
| $\theta_i = 0$             | Shared direction — subspaces overlap |
| $0 < \theta_i < \frac{\pi}{2}$ | Partial alignment |
| $\theta_i = \frac{\pi}{2}$ | Orthogonal in that direction |
</div></div>
Then:

- $P_u P_v$ has **eigenvalue $1$** when $\theta_i = 0$
- $P_u P_v$ has **eigenvalue $0$** when $\theta_i = \frac{\pi}{2}$

### 📌 Conclusion

- Principal angles provide a **canonical, coordinate-free measure** of subspace alignment.
- The matrix $P_u P_v$ **encodes these alignments** via its eigenvalues $\lambda_i = \cos^2(\theta_i)$
- These angles are used in applications like:
  - Subspace comparison
  - Canonical correlation analysis
  - Dimensionality reduction & projections

#### Proof

The proof requires several steps and starts with an analysis of the eigenproblem for $P_v P_u P_v$.

Let $(\lambda_i, v_i)$ be an eigenpair of $P_v P_u P_v$ with $\lambda_i > 0$ and $\Vert v_i \Vert = 1$,<br>
$\qquad$ and define $u_i$ as the normalized projection of $v_i$ onto $\mathcal{U}$:
$\;\;
u_i = \frac{P_u v_i}{\Vert P_u v_i \Vert}.
$

$\qquad$ Thus $\;v_i \in \mathcal{V},\;$ and $\;u_i \in \mathcal{U}$.

We seek to compute the angle between $u_i$ and $v_i$.
____

##### Step 1: Compute the Angle Between $u_i$ and $v_i$

Since $P_v P_u P_v$ maps into $\mathcal{V}$,
$\;
P_v P_u P_v v_i = \lambda_i v_i \quad \Rightarrow \quad v_i \in \mathcal{V} \quad \Rightarrow \quad P_v v_i = v_i.
$

From the eigenpair equation
$\;
\lambda_i v_i = P_v P_u P_v v_i = P_v P_u v_i,
\;$
thus $P_u v_i \neq 0$ when $\lambda_i \neq 0$ and $u_i$ is well defined.

We now compute the norm of $P_u v_i$<br>
$\qquad\begin{align}
\Vert P_u v_i \Vert^2
&= v_i^T P_u v_i \qquad\qquad\qquad &(\text{projection property: } P_u^2 = P_u) \\
&= (P_v v_i)^T P_u v_i              &(\text{since } P_v v_i = v_i) \\
&= v_i^T P_v P_u v_i                &(\text{since } P_v^T = P_v) \\
&= v_i^T P_v P_u P_v v_i            &(\text{again since } P_v = P_v^T) \\
&= \lambda_i v_i^T v_i              &(\text{from the eigenproblem}) \\
&= \lambda_i                        &(\text{since } \Vert v_i \Vert = 1).
\end{align}$

$\qquad$ Thus $\; \Vert P_u v_i \Vert = \sqrt{\lambda_i}$

____
We next compute the cosine of the angle $\theta_i$ between $u_i$ and $v_i$<br>
$\qquad\begin{align}
\cos \theta_i
&= \langle u_i, v_i \rangle \quad &(\text{since both are unit vectors}) \\
&= \frac{\langle P_u v_i, v_i \rangle}{\Vert P_u v_i \Vert} \qquad\qquad\qquad &(\text{definition of } u_i) \\
&= \frac{v_i^T P_u v_i}{\Vert P_u v_i \Vert} \quad &(\text{matrix form}) \\
&= \frac{(P_v v_i)^T P_u v_i}{\Vert P_u v_i \Vert} \quad &(\text{since } P_v v_i = v_i) \\
&= \frac{v_i^T P_v P_u v_i}{\Vert P_u v_i \Vert} \quad &(\text{symmetry of } P_v) \\
&= \frac{\lambda_i v_i^T v_i}{\Vert P_u v_i \Vert} \quad &(\text{eigenproblem again}) \\
&= \frac{\lambda_i}{\sqrt{\lambda_i}} \\
&= \sqrt{\lambda_i}.
\end{align}$

Thus
$\;
\cos^2 \theta_i = \lambda_i.
$

##### Step 2: Relation to the Singular Values of $M = U^T V$

Expand $P_v P_u P_v = V (V^T U)(U^T V) V^T = V (M^T M) V^T,\;
$
where
$\;
M = U^T V \in \mathbb{R}^{k \times \ell}.
$

$\qquad$ Note that $M$ encodes the inner products between orthonormal bases of $\mathcal{U}$ and $\mathcal{V}$.

Now consider the singular value decomposition (SVD)
$\;
M = A \Sigma B^T,
$<br>
$\qquad$ where $\Sigma = \operatorname{diag}(\sigma_1, \dotsc, \sigma_d)$, $d = \min(k, \ell)$.<br>
$\qquad$ Thus
$\;
M^T M = B \Sigma^T \Sigma B^T,
\;$
and the eigenvalues of $M^T M$ are $\sigma_i^2$.

Since
$\;
P_v P_u P_v = V (M^T M) V^T,
\;$
and $V$ has orthonormal columns, the nonzero eigenvalues $\lambda_i$ of $P_v P_u P_v$ are the eigenvalues of $M^T M$.

We therefore have
$\;
\lambda_i = \sigma_i^2,
\;$
and recalling that
$\;
\cos^2 \theta_i = \lambda_i,
$<br>
$\qquad$
we conclude
$\;
\cos \theta_i = \sigma_i.
$

---

##### Step 3: Conclusion

Finally, by standard properties of the singular value decomposition (SVD),<br>
$\qquad$ the singular values $\sigma_i$ correspond to the maximum cosines between directions in $\mathcal{U}$ and $\mathcal{V}$,<br>
under orthogonality constraints on previous choices.

Thus
- $\theta_i$ are the principal angles between $\mathcal{U}$ and $\mathcal{V}$,
- $u_i$ and $v_i$ are the corresponding principal vectors.

### 3.2.5 Symmetry of Principal Angles

<div style="background-color:#F2F5A9;color:black;padding-top:0.2cm;padding-bottom:0.2cm;">

**Thm:** (Principal Angles Are Symmetric)

<div style="padding-left:0.8cm;">
Let $\mathcal{U}, \mathcal{V} \subseteq \mathbb{R}^n$ be subspaces with orthonormal bases $U \in \mathbb{R}^{n \times k}$ and $V \in \mathbb{R}^{n \times \ell}$.

Form the matrix of inner products $M = U^T V \in \mathbb{R}^{k \times \ell}$.

Then:
- The singular values of $M$ and $M^T$ are identical.
- Therefore, the principal angles between $\mathcal{U}$ and $\mathcal{V}$ are the same as the principal angles between $\mathcal{V}$ and $\mathcal{U}$.

</div>
</div>

The singular values of a matrix $M$ are the square roots of the eigenvalues of $M^T M$ and $M M^T$.

Since $M \in \mathbb{R}^{k \times \ell}$:

- $M^T M$ is $\ell \times \ell$
- $M M^T$ is $k \times k$

Moreover, $M^T M$ and $M M^T$ share the same nonzero eigenvalues.
Thus, $M$ and $M^T$ have the same nonzero singular values.

$\qquad\therefore\quad$ The singular values of $U^T V$ and $V^T U$ are the same.

Since the principal angles are determined by these singular values, we conclude:

$\qquad\therefore\quad$ The principal angles between $\mathcal{U}$ and $\mathcal{V}$ are the same as between $\mathcal{V}$ and $\mathcal{U}$.

### 3.3 Summary: Computation of Pricipal Angles and Vectors

In practice, the principal angles and vectors between two subspaces can be computed by the following simple procedure:

<div style="float:left; padding-left:1cm;">

**Step-by-Step Summary**

| Step | Description |
|:---|:---|
| 1 | Choose orthonormal bases $U$ for $\mathcal{U}$ and $V$ for $\mathcal{V}$ |
| 2 | Form the matrix of inner products $M = U^T V$ |
| 3 | Compute the singular value decomposition (SVD) $M = A \Sigma B^T$ |
| 4 | Principal angles: $\cos(\theta_i) = \sigma_i$ |
| 5 | Principal vectors: $u_i = U a_i,\; v_i = V b_i$ |

</div>

The singular values $\sigma_i$ encode the cosines of the principal angles $\theta_i$,<br> and the left and right singular vectors $(a_i, b_i)$ specify the principal directions within the subspaces.

Thus, the geometry of the relationship between $\mathcal{U}$ and $\mathcal{V}$ is completely determined by the SVD of $U^T V$.

### 3.3.1 Example: Two 2D Hyperplanes in$\mathbb{R}^4$

### 3.3.1 Example: Subspaces with a Shared and an Orthogonal Direction

Consider two subspaces $\mathcal{U}, \mathcal{V} \subseteq \mathbb{R}^4$, each of dimension 2:

- $\mathcal{U}$ is spanned by orthonormal vectors $u_1, u_2$,
- $\mathcal{V}$ is spanned by $v_1, v_2$:
  - $v_1$ aligned with $u_1$ (perfect overlap),
  - $v_2$ orthogonal to both $u_1$ and $u_2$ (no overlap).

Thus, the expected principal angles are $\theta_1 = 0^\circ$, $\theta_2 = 90^\circ$.

In [64]:
%%julia
#----------------------------------------------------------
# Step 1: Define orthonormal basis for U
U = [1 0; 0 1; 0 0; 0 0]

# Define orthonormal basis for V
V = [1 0; 0 0; 0 0; 0 1]

display(py_show("Step 1:  ", L"\quad U =", U, L"\;\;V = ", V ))

#----------------------------------------------------------
# Step 2: Matrix of inner products
M = U' * V
display(py_show("Step 2:  ", L"\quad M = U^T V = ", M ))

#----------------------------------------------------------
# Step 3: Perform SVD
A, Σ, B = svd(M)
display(py_show("Step 3:  ", L"\quad ", "SVD(M) = ", Int.(A), Int.(Diagonal(Σ)), V ))

#----------------------------------------------------------
# Step 4: Principal Angles
θ = acosd.(Σ)

display(py_show( "Step 4:", L"\quad \cos(\theta_i) = ", Int.(Σ)', L",\qquad", "Principal angles (in degrees): ", Int.(θ)'))

#----------------------------------------------------------
# Step 5: Principal Vectors
u₁ = Int.(U * A[:, 1])
u₂ = Int.(U * A[:, 2])

v₁ = Int.(V * B[:, 1])
v₂ = Int.(V * B[:, 2])

display(py_show("Step 5:"))
display(py_show(L"\qquad ", "First vector pair: ", u₁, v₁, L"\quad", " principal angle ", θ[1]))
display(py_show(L"\qquad ", "First vector pair: ", u₂, v₂, L"\quad", " principal angle ", θ[2]))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

**Interpretation:**

- The first principal vectors $u_1$ and $v_1$ are perfectly aligned, corresponding to a principal angle of $0^\circ$.
- The second principal vectors $u_2$ and $v_2$ are fully orthogonal, corresponding to a principal angle of $90^\circ$.
- This confirms the expected geometric relationship between $\mathcal{U}$ and $\mathcal{V}$.


### 3.3.2 Example: Partial Overlap with Zero Singular Values

We define two subspaces:

- $\mathcal{U}$ of dimension 2,
- $\mathcal{V}$ of dimension 3,
- both in $\mathbb{R}^4$.

In [66]:
%%julia

# ------------------------------------------------------
# Step 1: Define orthonormal basis for U (2D)
U = [1 0;
     0 1;
     0 0;
     0 0]

# Define V to have partial overlap with U
θ = 30 * (π/180)  # 30 degrees in radians
c = cos(θ)
s = sin(θ)

v1 = [c; s; 0; 0] # v₁ is tilted 30 degrees relative to u₁
v2 = [0; 0; 1; 0] # v₂ orthogonal to u₁, u₂
v3 = [0; 0; 0; 1] # v₃ orthogonal to everything above
V  = hcat(v1, v2, v3)

# ----------------------------------------------------------------------
# Step 2: Matrix of inner products
M = U' * V
# ----------------------------------------------------------------------
# Step 3: Perform SVD
A, Σ, B = svd(M)
# ----------------------------------------------------------------------
# Step 4: Principal angles
θ = acosd.(Σ)

# ----------------------------------------------------------------------
# Step : Principal vectors
u₁ = U * A[:, 1]
v₁ = V * B[:, 1]

u₂ = U * A[:, 2]
v₂ = V * B[:, 2]

# ----------------------------------------------------------------------
# Results
display(py_show("Step 1:  ", L"\quad U =", U, L"\;\;V = ", V ))
display(py_show("Step 2:  ", L"\quad M = U^T V = ", M ))
display(py_show("Step 3:  ", L"\quad ", "SVD(M) = ", A, Diagonal(Σ), V ))
display(py_show("Step 4:", L"\quad \cos(\theta_i) = ", Int.(Σ)', L",\qquad", "Principal angles (in degrees): ", Int.(θ)'))


Matrix of inner products M = Uᵀ V:

Singular values (cosines of principal angles):

Principal angles (in degrees):

First principal vector pair (u₁, v₁):

Second principal vector pair (u₂, v₂):


## X.x Angle Between Subspaces

#### Angle Between Two Subspaces and Principal Angles

Let $\mathcal{U}, \mathcal{V} \subseteq \mathbb{R}^n$ be two $k$-dimensional subspaces. To describe their relative orientation, we define the **principal angles** $\theta_1, \dots, \theta_k$ between them.

Assuming we have orthonormal bases:

- $U \in \mathbb{R}^{n \times k}$ for $\mathcal{U}$
- $V \in \mathbb{R}^{n \times k}$ for $\mathcal{V}$

We compute the matrix of inner products:

$\qquad
M = U^T V,
$

and perform a singular value decomposition (SVD):

$\qquad
M = A \Sigma B^T, \quad \Sigma = \mathrm{diag}(\sigma_1, \dots, \sigma_k).
$

The **principal angles** are:

$\qquad
\theta_i = \cos^{-1}(\sigma_i), \quad \text{for } i = 1, \dots, k.
$

---

#### Example: Two 2D Subspaces in $\mathbb{R}^3$

Let $\mathcal{U}, \mathcal{V} \subseteq \mathbb{R}^3$ be 2-dimensional subspaces with orthonormal bases:

- $\mathcal{U}$: $\{u_1, u_2\}$
- $\mathcal{V}$: $\{v_1, v_2\}$

The principal angles are then directly computed from dot products:

$\qquad
\theta_1 = \cos^{-1} \left( \frac{\langle u_1, v_1 \rangle}{\|u_1\| \|v_1\|} \right), \qquad
\theta_2 = \cos^{-1} \left( \frac{\langle u_2, v_2 \rangle}{\|u_2\| \|v_2\|} \right).
$

These angles quantify how closely the two planes align, ranging from $0^\circ$ (perfect alignment) to $90^\circ$ (orthogonal subspaces).

#### Formal Computation

Assume we have orthonormal bases:

- $U \in \mathbb{R}^{n \times k}$ for $\mathcal{U}$
- $V \in \mathbb{R}^{n \times k}$ for $\mathcal{V}$

1. Compute the matrix of inner products:

$\qquad
M = U^T V
$

2. Perform a singular value decomposition (SVD):

$\qquad
M = A \Sigma B^T, \quad \Sigma = \mathrm{diag}(\sigma_1, \dots, \sigma_k)
$

3. The **principal angles** are given by:

$\qquad
\theta_i = \cos^{-1}(\sigma_i), \quad \text{for } i = 1, \dots, k
$

Each angle lies in the range $[0^\circ, 90^\circ]$:
- $\theta_i = 0^\circ$ means the subspaces overlap in that direction
- $\theta_i = 90^\circ$ means they are orthogonal in that direction

---

##### Example: Two 2D Subspaces in $\mathbb{R}^3$

Let $\mathcal{U}, \mathcal{V} \subseteq \mathbb{R}^3$ be two 2D subspaces with orthonormal bases:

- $\mathcal{U}$: $\{u_1, u_2\}$
- $\mathcal{V}$: $\{v_1, v_2\}$

Then:

$\qquad
\theta_1 = \cos^{-1} \left( \frac{\langle u_1, v_1 \rangle}{\|u_1\| \|v_1\|} \right), \qquad
\theta_2 = \cos^{-1} \left( \frac{\langle u_2, v_2 \rangle}{\|u_2\| \|v_2\|} \right)
$

These two angles describe the full geometric relationship between the planes.
____

When comparing two subspaces, especially of the same dimension, the notion of an angle generalizes beyond the case of a single vector and a plane.

Given two $k$-dimensional subspaces $\mathcal{U}, \mathcal{V} \subseteq \mathbb{R}^n$, the angle between them is not a single scalar, but a set of angles known as **principal angles**.

Intuitively, the smallest principal angle measures how close the two subspaces are to overlapping along a common direction.
* If two subspaces share a common vector direction, the smallest angle will be $0^\circ$.
* If they are orthogonal, the largest principal angle will be $90^\circ$.

The first principal angle $\theta_1$ is defined as the smallest angle between any two unit vectors $u \in \mathcal{U}$ and $v \in \mathcal{V}$:

$\qquad
\cos(\theta_1) = \max_{u \in \mathcal{U}, \ v \in \mathcal{V}} \langle u, v \rangle \quad \text{subject to} \quad \|u\| = \|v\| = 1.
$

Subsequent principal angles are defined recursively, constraining the directions to be orthogonal to all previously chosen ones.<br>
These angles give a complete geometric relationship between the two subspaces.

### 2.3.1 Computing Principal Angles Between Subspaces

Let $\mathcal{U}$ and $\mathcal{V}$ be two 2-dimensional subspaces of $\mathbb{R}^3$.

We define **orthonormal bases** for each subspace:
- For $\mathcal{U}$: $\{u_1, u_2\}$
- For $\mathcal{V}$: $\{v_1, v_2\}$

The **principal angles** $\theta_1$ and $\theta_2$ between the subspaces are defined recursively as the smallest angles between pairs of unit vectors:

- $\theta_1$ is the smallest angle between any $u \in \mathcal{U}$ and $v \in \mathcal{V}$
- $\theta_2$ is the next smallest angle between directions **orthogonal to the first pair**

In this 2D case with known bases, the principal angles are computed directly as:

$\qquad
\cos(\theta_1) = \frac{\langle u_1, v_1 \rangle}{\|u_1\| \|v_1\|}, \quad
\theta_1 = \cos^{-1} \left( \frac{\langle u_1, v_1 \rangle}{\|u_1\| \|v_1\|} \right)
$

$\qquad
\cos(\theta_2) = \frac{\langle u_2, v_2 \rangle}{\|u_2\| \|v_2\|}, \quad
\theta_2 = \cos^{-1} \left( \frac{\langle u_2, v_2 \rangle}{\|u_2\| \|v_2\|} \right)
$

This approach uses the fact that:
- $u_1, u_2$ form an orthonormal basis for $\mathcal{U}$
- $v_1, v_2$ are obtained by rotating and tilting the basis vectors of $\mathcal{U}$ into $\mathcal{V}$

The computed angles $\theta_1$ and $\theta_2$ describe the geometric alignment between the subspaces.

### 2.3.2 Interactive Figure of Two Planes in 3D

### 2.3.4 Computing Principal Angles from Arbitrary Bases

Let $\mathcal{U}, \mathcal{V} \subset \mathbb{R}^n$ be two $k$-dimensional subspaces, each defined by a basis:

- $A \in \mathbb{R}^{n \times k}$: a matrix whose columns form a basis for $\mathcal{U}$
- $B \in \mathbb{R}^{n \times k}$: a matrix whose columns form a basis for $\mathcal{V}$

These bases do **not** need to be orthonormal.

#### Step 1: Orthonormalize the Bases

Begin by orthonormalizing the columns of $A$ and $B$ using QR decomposition:

$\qquad
A = Q_U R_U, \quad B = Q_V R_V
$

where $Q_U, Q_V \in \mathbb{R}^{n \times k}$ have orthonormal columns that span $\mathcal{U}$ and $\mathcal{V}$ respectively.

#### Step 2: Compute the Matrix of Inner Products

Compute the matrix of cosines between the orthonormal bases:

$\qquad
M = Q_U^T Q_V \in \mathbb{R}^{k \times k}
$

#### Step 3: Singular Value Decomposition

The **principal angles** are obtained from the singular values of $M$:

$\qquad
M = U \Sigma V^T, \quad \Sigma = \mathrm{diag}(\sigma_1, \dots, \sigma_k)
$

Then the principal angles are:

$\qquad
\theta_i = \cos^{-1}(\sigma_i), \quad \text{for } i = 1, \dots, k
$

Each $\theta_i \in [0^\circ, 90^\circ]$ measures how aligned the $i$-th principal direction of $\mathcal{U}$ is with that of $\mathcal{V}$.

____
**Remark:** This procedure generalizes the concept of the angle between two vectors to higher-dimensional subspaces.

## 2.4 Projection Matrices and Subspace Intersections

The intersection of two subspaces can be investigated using their projection matrices:
- Let `P = UUᵀ` and `Q = VVᵀ` be orthogonal projections.
- Then the intersection of the subspaces corresponds to the **invariant subspace** of `PQ` or `QP`.

The angle between subspaces is also encoded in the eigenvalues of `PQ`:
- If `PQ` has an eigenvalue of `1`, it corresponds to a shared direction.
- Singular values of `UᵀV` relate to `cos(θᵢ)` for principal angles.

This builds geometric insight into the SVD structure used in computing principal directions.

The concept of principal angles offers a natural way to compare subspaces of equal dimension. For two $k$-dimensional subspaces $\mathcal{U}, \mathcal{V} \subseteq \mathbb{R}^n$, there are $k$ principal angles $\theta_1, \dots, \theta_k$ defined recursively.

Each angle $\theta_i$ captures the cosine of the largest possible inner product between unit vectors $u_i \in \mathcal{U}$ and $v_i \in \mathcal{V}$, under the constraint that $u_i$ and $v_i$ are orthogonal to the previously selected vectors:

$$
\cos(\theta_i) = \max_{u \in \mathcal{U}, \ v \in \mathcal{V}} \langle u, v \rangle, \quad \text{subject to} \quad \|u\| = \|v\| = 1, \ u \perp u_j, \ v \perp v_j \ \text{for } j < i.
$$

The collection of angles describes how the subspaces align:
- If all $\theta_i = 0$, the subspaces are identical.
- If all $\theta_i = 90^\circ$, the subspaces are orthogonal.
- Intermediate values reflect partial alignment.

These angles are closely related to the singular values of $U^T V$, where $U$ and $V$ are orthonormal bases for $\mathcal{U}$ and $\mathcal{V}$ respectively.

## Principal Angle Visualizer

The following demo illustrates principal directions and angles for two 3D planes intersecting in a line.

- **Plane U (dark)**: fixed in the XY plane.
- **Plane V (light)**: defined by azimuth (`θ₁`) and elevation (`θ₂`).
  - `θ₁`: rotates Plane V about Z-axis (in-plane direction).
  - `θ₂`: tilts Plane V upward (out-of-plane direction).
- **Principal Directions**:
  - First pair (`red`): aligned with line of intersection.
  - Second pair (`green`): orthogonal in-plane directions.
- **Arcs** (`θ₁ arc`, `θ₂ arc`): show angle between principal directions.
- **Intersection Line**: dashed black line — true geometric overlap.

In [ ]:
# Define original plane colors
# Define original plane colors
PLANE1_COLOR = '#1f77b4'  # original blue
PLANE2_COLOR = '#ff7f0e'  # original orange

# New color for VectorAngleViewer
VECTOR1_COLOR = '#1f77b4'  # blue
VECTOR2_COLOR = '#ff7f0e'  # orange

def compute_principal_vectors_and_angles(Q1, Q2):
    """
    Computes principal vectors and principal angles between two subspaces spanned by Q1 and Q2.
    Assumes Q1 and Q2 have orthonormal columns.
    Returns principal vectors for Q1, principal vectors for Q2, and angles in degrees.
    """
    M = np.dot(Q1.T, Q2)
    U, S, Vh = np.linalg.svd(M)
    S = np.clip(S, -1.0, 1.0)  # Protect against numerical drift outside [-1, 1]
    principal_U = np.dot(Q1, U)
    principal_V = np.dot(Q2, Vh.T)
    angles = np.degrees(np.arccos(S))
    return principal_U, principal_V, angles

In [ ]:
hv.extension('bokeh')
viewer = VectorAngleViewer()
pn.panel(viewer).servable()

In [ ]:
hv.extension('bokeh')
class SubspacePlotter2D(pn.viewable.Viewer):
    Q1 = param.Array()
    Q2 = param.Array()
    vector_scale = param.Number(1.0, bounds=(0.1, 5.0))

    def __init__(self, **params):
        super().__init__(**params)

    @pn.depends('Q1', 'Q2', 'vector_scale')
    def plot(self):
        segments = []
        for i in range(self.Q1.shape[1]):
            segments.append((0, 0, self.vector_scale*self.Q1[0, i], self.vector_scale*self.Q1[1, i]))
        for i in range(self.Q2.shape[1]):
            segments.append((0, 0, self.vector_scale*self.Q2[0, i], self.vector_scale*self.Q2[1, i]))
        labels = [f"Q1 Vec {i+1}" for i in range(self.Q1.shape[1])] + [f"Q2 Vec {i+1}" for i in range(self.Q2.shape[1])]
        colors = [PLANE1_COLOR]*self.Q1.shape[1] + [PLANE2_COLOR]*self.Q2.shape[1]

        data = [hv.Segments([seg]).opts(color=col, line_width=3) for seg, col in zip(segments, colors)]
        overlay = hv.Overlay(data).opts(width=500, height=500, xlim=(-1, 1), ylim=(-1, 1), aspect='equal', title="2D Subspaces")
        return overlay

def plot_subspaces_2d(Q1, Q2):
    """
    Simple functional wrapper for 2D subspace plot.
    """
    return SubspacePlotter2D(Q1=Q1, Q2=Q2).plot()


In [ ]:
hv.extension('plotly')
class SubspacePlotter3D(pn.viewable.Viewer):
    Q1 = param.Array()
    Q2 = param.Array()
    vector_scale = param.Number(1.0, bounds=(0.1, 5.0))

    def __init__(self, **params):
        super().__init__(**params)

    @pn.depends('Q1', 'Q2', 'vector_scale')
    def plot(self):
        fig = go.Figure()
        for i in range(self.Q1.shape[1]):
            fig.add_trace(go.Scatter3d(
                x=[0, self.vector_scale*self.Q1[0, i]],
                y=[0, self.vector_scale*self.Q1[1, i]],
                z=[0, self.vector_scale*self.Q1[2, i]],
                mode='lines+markers',
                name=f'Q1 Vec {i+1}',
                line=dict(color=PLANE1_COLOR, width=5)
            ))
        for i in range(self.Q2.shape[1]):
            fig.add_trace(go.Scatter3d(
                x=[0, self.vector_scale*self.Q2[0, i]],
                y=[0, self.vector_scale*self.Q2[1, i]],
                z=[0, self.vector_scale*self.Q2[2, i]],
                mode='lines+markers',
                name=f'Q2 Vec {i+1}',
                line=dict(color=PLANE2_COLOR, width=5)
            ))

        fig.update_layout(
            width=700, height=700,
            scene=dict(
                xaxis=dict(range=[-1, 1]),
                yaxis=dict(range=[-1, 1]),
                zaxis=dict(range=[-1, 1])
            ),
            title='3D Subspaces'
        )

        return pn.panel(fig)



def plot_subspaces_3d(Q1, Q2):
    """
    Simple functional wrapper for 3D subspace plot.
    """
    return SubspacePlotter3D(Q1=Q1, Q2=Q2).plot()

## Organization of Sections

| Section | Title | Goal / Content |
|:---|:---|:---|
| **3.2** | **Principal Angles via Projections** | Introduce **why** we study \( P_u P_v \). Motivation: to relate projections and angles between subspaces. Define \( P_u = UU^T \), \( P_v = VV^T \). |
| **3.2.1** | **First Properties of \( P_u P_v \)** | Explore properties of \( P_u P_v \): mapping, non-symmetry, ranks. Show simple examples. **(Proofs for basic properties.)** |
| **3.2.2** | **Symmetry of Principal Angles** | Establish that angles between \( \mathcal{U} \) and \( \mathcal{V} \) are symmetric: study \( P_v P_u \) and its singular values. **(Short proof.)** |
| **3.2.3** | **Behavior of \( P_v P_u P_v \)** | Study \( P_v P_u P_v \). Show how it relates to principal angles: eigenvalues relate to \( \cos^2(\theta_i) \). **(Proof of connection.)** |
| **3.3** | **Principal Vectors and SVD** | Transition to SVD. Show that singular values of \( U^T V \) give \( \cos(\theta_i) \). Define principal vectors formally. **(Proof connecting SVD to principal vectors.)** |
| ?? | (Postponed) Symmetry of $P_u P_v$ and $P_v P_u$ | |
| **3.4** | **Summary and Key Takeaways** | Collect results: multiple views (projection products, SVD, eigenvalues). Overview of main formulas and theorems. Prepare for applications. |

---

## Proof Placement

| Location | Proof content |
|:---|:---|
| 3.2.1 | Basic properties: non-symmetry of \( P_u P_v \), projection behaviors. |
| 3.2.2 | Symmetry of principal angles: nonzero singular values match. |
| 3.2.3 | Eigenvalues of \( P_v P_u P_v \) are \( \cos^2(\theta_i) \). |
| 3.3 | Formal derivation: principal vectors from SVD of \( U^T V \). |

---

## Logical Flow Diagram

```mermaid
flowchart TD
    A(Principal Angles via Projections) --> B(First Properties of P_u P_v)
    B --> C(Symmetry: P_u P_v vs P_v P_u)
    C --> D(Behavior of P_v P_u P_v)
    D --> E(Principal Vectors via SVD)
    E --> F(Summary and Key Takeaways)


## 3.2 Projections and Subspace Interaction
- Introduce projections $P_u = UU^T$ and $P_v = VV^T$.
- Motivate the study of the composition $P_u P_v$.
- Set up the framework: angles between $P_v x$ and $P_u P_v x$.

## 3.3 Properties of the Projection Composition
- Explore basic properties of $P_u P_v$:
  - Rank properties.
  - Symmetry/Non-symmetry.
  - Behavior when $\mathcal{U} = \mathcal{V}$ or $\mathcal{U} \perp \mathcal{V}$.
- Prepare for spectral study.

## 3.4 Principal Angles via Singular Values
- Formally introduce the principal angles $\theta_1, \dots, \theta_d$.
- Define $M = U^T V$.
- Show that the singular values of $M$ are $\cos(\theta_i)$.
- Insert **NEW small proof**: eigenvalues of $P_u P_v$ are $\cos^2(\theta_i)$.
- Interpret principal vectors.
### 3.4.1 Connection Between $P_u P_v$ and Principal Angles

We now establish the connection between the eigenvalues of $P_u P_v$ and the principal angles between $\mathcal{U}$ and $\mathcal{V}$.

Let:
- $U \in \mathbb{R}^{n \times k}$ and $V \in \mathbb{R}^{n \times \ell}$ be orthonormal bases,
- $d = \min(k, \ell)$,
- $M = U^T V \in \mathbb{R}^{k \times \ell}$.

Then:
- The singular values of $M$ are $\sigma_1, \dots, \sigma_d$,
- The **principal angles** $\theta_i$ are given by $\cos(\theta_i) = \sigma_i$.

Now observe:
- $P_u = U U^T$, $\quad$ $P_v = V V^T$,
- Hence, $\quad P_u P_v = U (U^T V) (V^T)$.

Let $M = A \Sigma B^T$ be the singular value decomposition of $M$, with $\Sigma = \mathrm{diag}(\sigma_1, \dots, \sigma_d)$.

Thus:

$\qquad
P_u P_v = (U A) \Sigma (V B)^T.
$

Since $U A$ and $V B$ have orthonormal columns, this is a "partial SVD" of $P_u P_v$.

Therefore:
- The **nonzero eigenvalues** of $P_u P_v$ are $\cos^2(\theta_1), \dotsc, \cos^2(\theta_d)$.

---

**Conclusion:**  
The spectrum of $P_u P_v$ directly encodes the principal angles between $\mathcal{U}$ and $\mathcal{V}$.


## 3.5 Symmetry Between Subspaces (Optional/Deferred)
- Discuss $P_v P_u$ and its relationship to $P_u P_v$.
- Establish that angles between $\mathcal{U}$ and $\mathcal{V}$ are symmetric.

## 3.6 Examples and Visualization
- Simple 2D examples: intersecting lines, planes.
- Interactive visualizations of principal angles.
